In [33]:
#setup dependancies
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import requests
import gmaps
import os

from api_keys import gmap_key

In [34]:
df = pd.read_csv("./data/cities.csv")
df.head

<bound method NDFrame.head of                  City  Latitude  Longitude  Max Temperature  Humidity  \
0              Luanda   -8.8368    13.2343            84.20        74   
1           Mahébourg  -20.4081    57.7000            82.69        65   
2                 Itō   34.9667   139.0833            37.69        57   
3           Namatanai   -3.6667   152.4333            80.04        81   
4    Praia da Vitória   38.7333   -27.0667            59.76        82   
..                ...       ...        ...              ...       ...   
574             Zabid   14.1951    43.3152            86.85        42   
575              Atar   20.5169   -13.0499            77.70        12   
576      Taitung City   22.7583   121.1444            57.47        82   
577         Tucumcari   35.1717  -103.7250            31.95       100   
578          Meulaboh    4.1363    96.1285            78.06        87   

     Cloudiness  Wind Speed Nation        Date  
0            40        4.61     AO  16745574

In [35]:
#configure gmaps
gmaps.configure(api_key = gmap_key)
locations = df[["Latitude", "Longitude"]]
weight = df["Humidity"]

#heatmap layer
fig = gmaps.figure(center = (0, 0), zoom_level = 1)
heatmap_layer = gmaps.heatmap_layer(locations, weights = weight)
fig.add_layer(heatmap_layer)
fig

Figure(layout=FigureLayout(height='420px'))

In [36]:
#ideal conditions filter
temp_optimal = (df["Max Temperature"] <= 80) & (df["Max Temperature"] >= 70)
wind_optimal = (df["Wind Speed"] <= 10)
cloud_optimal = (df["Cloudiness"] <= 0)
humid_optimal = (df["Humidity"] <= 30)
weather_optimal = df.loc[temp_optimal & wind_optimal & cloud_optimal & humid_optimal]

weather_optimal = weather_optimal.dropna()

weather_optimal

,City,Latitude,Longitude,Max Temperature,Humidity,Cloudiness,Wind Speed,Nation,Date
20,Taoudenni,22.6783,-3.9836,72.86,15,0,6.96,ML,1674557702
116,Adrar,20.5022,-10.0711,73.02,12,0,9.08,MR,1674557714
133,Tessalit,20.1986,1.0114,79.38,10,0,7.20,ML,1674557716


In [41]:
# store into variable name hotel_df
hotel_df = weather_optimal

hotel_name = []


<class 'pandas.core.frame.DataFrame'>
Int64Index: 3 entries, 20 to 133
Data columns (total 10 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   City             3 non-null      object 
 1   Latitude         3 non-null      float64
 2   Longitude        3 non-null      float64
 3   Max Temperature  3 non-null      float64
 4   Humidity         3 non-null      int64  
 5   Cloudiness       3 non-null      int64  
 6   Wind Speed       3 non-null      float64
 7   Nation           3 non-null      object 
 8   Date             3 non-null      int64  
 9   Hotel Name       3 non-null      object 
dtypes: float64(4), int64(3), object(3)
memory usage: 372.0+ bytes


,City,Latitude,Longitude,Max Temperature,Humidity,Cloudiness,Wind Speed,Nation,Date,Hotel Name
20,Taoudenni,22.6783,-3.9836,72.86,15,0,6.96,ML,1674557702,
116,Adrar,20.5022,-10.0711,73.02,12,0,9.08,MR,1674557714,
133,Tessalit,20.1986,1.0114,79.38,10,0,7.20,ML,1674557716,


In [53]:
for index, row in hotel_df.iterrows():
    lat = row["Latitude"]
    lng = row["Longitude"]
    city = row["City"]
    coords = (f"{lat}, {lng}")

    parameters = {
    "key": gmap_key,
    # Set parameters to search for hotels with 5000 meters
    "radius": 5000,
    "location": coords,
    "type": "lodging"
    }

    url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    response_json = requests.get(url, params = parameters).json()

    print(response_json)
    
    #regardless of radius, "ZERO_RESULTS" Will appear. Can not find a fix. Therefore, I can not add the hotel markers to the heatmap.
    



{'html_attributions': [], 'results': [], 'status': 'ZERO_RESULTS'}
{'html_attributions': [], 'results': [], 'status': 'ZERO_RESULTS'}
{'html_attributions': [], 'results': [], 'status': 'ZERO_RESULTS'}
